In [1]:
import json
import os
import re
from pathlib import Path
import numpy as np
import cv2
import copy
import random
import xml.etree.ElementTree as ET
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from pycocotools.coco import COCO

### test 1, 2, 3, 4

In [3]:
cls_names = ['cube', 'can', 'box', 'bottle']
clsname2ind = {'cube':1, 'can':2, 'box':3, 'bottle':4}
# create category
data_categories = []
for cat in cls_names:   
    cat_id  = clsname2ind[cat]
    dic = {}
    dic['supercategory'] = 'None'
    dic['id'] = cat_id
    dic['name'] = cat
    data_categories.append(dic)
    
cls1_inds = range(50)
cls2_inds = range(50, 100)
cls3_inds = range(100, 150)
cls4_inds = range(150, 200)

In [7]:
data_root = '/home/tony/YCB_simulation/query'
im_dir = os.path.join(data_root, 'images')
nd_dir = os.path.join(data_root, 'ndarray')
ann_cnt = 0
test_images = []
test_annotations = []

for n, inds in enumerate([cls1_inds, cls2_inds, cls3_inds, cls4_inds]):
    target_cls_id = n + 1
    for i in inds:
        # im
        im_file_name = str(i).zfill(6) + '.jpg'
        dic = {}
        dic['license'] = 1
        dic['file_name'] = im_file_name
        dic['coco_url'] = 'http://farm3.staticflickr.com/2253/1755223462_fabbeb8dc3_z.jpg'
        dic['height'] = 256
        dic['width'] = 256
        dic['date_captured'] = '2013-11-15 13:55:22'
        dic['id'] = i
        test_images.append(dic)
        # ann
        ann_file_name = str(i).zfill(6) + '.npy'
        boxes = np.load(os.path.join(nd_dir, ann_file_name), allow_pickle=True)
        for j in range(boxes.shape[0]):
            box = boxes[j]
            cls_id = int(box[4])
            if cls_id != target_cls_id:
                continue
            ann_dic = {}
            ann_dic['segmentation'] = [[184.05]]
            ann_dic['area'] = 1.28
            ann_dic['iscrowd'] = 0
            ann_dic['image_id'] = i
            ann_dic['bbox'] = [int(box[0]), int(box[1]), int(box[2]) - int(box[0]), int(box[3]) - int(box[1])]
            ann_dic['category_id'] = cls_id
            ann_dic['id'] = int(str(i)+str(j))
            test_annotations.append(ann_dic)

coco_json_path = '/home/tony/datasets/coco/annotations/instances_minival2014.json'
with open(coco_json_path, 'r') as f:
    data = json.load(f)
_info = data['info']
_licenses = data['licenses']

new_dict = {}
new_dict['info'] = _info
new_dict['images'] = test_images
new_dict['licenses'] = _licenses
new_dict['annotations'] = test_annotations
new_dict['categories'] = data_categories
dump_path = f'/home/tony/datasets/YCB2D/annotations/instances_inferencefs.json'
with open(dump_path, 'w') as f:
    json.dump(new_dict, f)

In [8]:
_COCO = COCO(dump_path)

print(len(_COCO.imgs))
print(len(_COCO.anns))
print(len(_COCO.cats))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
200
1891
4


### oracle test set

In [35]:
data_root = '/home/tony/YCB_simulation/query'
im_dir = os.path.join(data_root, 'images')
nd_dir = os.path.join(data_root, 'ndarray')
ann_cnt = 0
test_images = []
test_annotations = []

for i in range(200):
    # im
    im_file_name = str(i).zfill(6) + '.jpg'
    dic = {}
    dic['license'] = 1
    dic['file_name'] = im_file_name
    dic['coco_url'] = 'http://farm3.staticflickr.com/2253/1755223462_fabbeb8dc3_z.jpg'
    dic['height'] = 256
    dic['width'] = 256
    dic['date_captured'] = '2013-11-15 13:55:22'
    dic['id'] = i
    test_images.append(dic)
    # ann
    ann_file_name = str(i).zfill(6) + '.npy'
    boxes = np.load(os.path.join(nd_dir, ann_file_name), allow_pickle=True)
    for j in range(boxes.shape[0]):
        box = boxes[j]
        cls_id = int(box[4])
        dic = {}
        dic['segmentation'] = [[184.05]]
        dic['area'] = 1.28
        dic['iscrowd'] = 0
        dic['image_id'] = i
        dic['bbox'] = [int(box[0]), int(box[1]), int(box[2]) - int(box[0]), int(box[3]) - int(box[1])]
        dic['category_id'] = cls_id
        dic['id'] = ann_cnt
        ann_cnt += 1
        test_annotations.append(dic)

In [36]:
coco_json_path = '/home/tony/datasets/coco/annotations/instances_minival2014.json'
with open(coco_json_path, 'r') as f:
    data = json.load(f)
_info = data['info']
_licenses = data['licenses']

new_dict = {}
new_dict['info'] = _info
new_dict['images'] = test_images
new_dict['licenses'] = _licenses
new_dict['annotations'] = test_annotations
new_dict['categories'] = data_categories
dump_path = '/home/tony/datasets/YCB2D/annotations/instances_inference1234.json'
with open(dump_path, 'w') as f:
    json.dump(new_dict, f)

In [37]:
_COCO = COCO(dump_path)

print(len(_COCO.imgs))
print(len(_COCO.anns))
print(len(_COCO.cats))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
200
2563
4


### Dense set

In [16]:
cls_names = ['cube', 'can', 'box', 'bottle']
clsname2ind = {'cube':1, 'can':2, 'box':3, 'bottle':4}
# create category
data_categories = []
for cat in cls_names:   
    cat_id  = clsname2ind[cat]
    dic = {}
    dic['supercategory'] = 'None'
    dic['id'] = cat_id
    dic['name'] = cat
    data_categories.append(dic)
cls_im_inds = [list(range(200, 250)), list(range(250, 300)), list(range(300, 350)), list(range(350, 400))]
data_root = '/home/tony/YCB_simulation/query'
im_dir = os.path.join(data_root, 'images')
nd_dir = os.path.join(data_root, 'ndarray')
test_images = []
test_annotations = []

for cls, inds in zip(cls_names, cls_im_inds):
    target_cls_id = clsname2ind[cls]
    for i in inds:
        # im
        im_file_name = str(i).zfill(6) + '.jpg'
        dic = {}
        dic['license'] = 1
        dic['file_name'] = im_file_name
        dic['coco_url'] = 'http://farm3.staticflickr.com/2253/1755223462_fabbeb8dc3_z.jpg'
        dic['height'] = 256
        dic['width'] = 256
        dic['date_captured'] = '2013-11-15 13:55:22'
        dic['id'] = i
        test_images.append(dic)
        # ann
        ann_file_name = str(i).zfill(6) + '.npy'
        boxes = np.load(os.path.join(nd_dir, ann_file_name), allow_pickle=True)
        for j in range(boxes.shape[0]):
            box = boxes[j]
            cls_id = int(box[4])
            if cls_id != target_cls_id:
                continue
            dic = {}
            dic['segmentation'] = [[184.05]]
            dic['area'] = 1.28
            dic['iscrowd'] = 0
            dic['image_id'] = i
            dic['bbox'] = [int(box[0]), int(box[1]), int(box[2]) - int(box[0]), int(box[3]) - int(box[1])]
            dic['category_id'] = cls_id
            dic['id'] = int(str(i)+str(j))
            test_annotations.append(dic)
coco_json_path = '/home/tony/datasets/coco/annotations/instances_minival2014.json'
with open(coco_json_path, 'r') as f:
    data = json.load(f)
_info = data['info']
_licenses = data['licenses']

new_dict = {}
new_dict['info'] = _info
new_dict['images'] = test_images
new_dict['licenses'] = _licenses
new_dict['annotations'] = test_annotations
new_dict['categories'] = data_categories
dump_path = '/home/tony/datasets/YCB2D/annotations/instances_inferencefsdense.json'
with open(dump_path, 'w') as f:
    json.dump(new_dict, f)

In [17]:
_COCO = COCO(dump_path)

print(len(_COCO.imgs))
print(len(_COCO.anns))
print(len(_COCO.cats))

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
200
3694
4


In [18]:
cls_names = ['cube', 'can', 'box', 'bottle']
clsname2ind = {'cube':1, 'can':2, 'box':3, 'bottle':4}
# create category
data_categories = []
for cat in cls_names:   
    cat_id  = clsname2ind[cat]
    dic = {}
    dic['supercategory'] = 'None'
    dic['id'] = cat_id
    dic['name'] = cat
    data_categories.append(dic)
cls_im_inds = [list(range(200, 250)), list(range(250, 300)), list(range(300, 350)), list(range(350, 400))]
data_root = '/home/tony/YCB_simulation/query'
im_dir = os.path.join(data_root, 'images')
nd_dir = os.path.join(data_root, 'ndarray')
test_images = []
test_annotations = []

for cls, inds in zip(cls_names, cls_im_inds):
    for i in inds:
        # im
        im_file_name = str(i).zfill(6) + '.jpg'
        dic = {}
        dic['license'] = 1
        dic['file_name'] = im_file_name
        dic['coco_url'] = 'http://farm3.staticflickr.com/2253/1755223462_fabbeb8dc3_z.jpg'
        dic['height'] = 256
        dic['width'] = 256
        dic['date_captured'] = '2013-11-15 13:55:22'
        dic['id'] = i
        test_images.append(dic)
        # ann
        ann_file_name = str(i).zfill(6) + '.npy'
        boxes = np.load(os.path.join(nd_dir, ann_file_name), allow_pickle=True)
        for j in range(boxes.shape[0]):
            box = boxes[j]
            cls_id = int(box[4])
            dic = {}
            dic['segmentation'] = [[184.05]]
            dic['area'] = 1.28
            dic['iscrowd'] = 0
            dic['image_id'] = i
            dic['bbox'] = [int(box[0]), int(box[1]), int(box[2]) - int(box[0]), int(box[3]) - int(box[1])]
            dic['category_id'] = cls_id
            dic['id'] = int(str(i)+str(j))
            test_annotations.append(dic)
coco_json_path = '/home/tony/datasets/coco/annotations/instances_minival2014.json'
with open(coco_json_path, 'r') as f:
    data = json.load(f)
_info = data['info']
_licenses = data['licenses']

new_dict = {}
new_dict['info'] = _info
new_dict['images'] = test_images
new_dict['licenses'] = _licenses
new_dict['annotations'] = test_annotations
new_dict['categories'] = data_categories
dump_path = '/home/tony/datasets/YCB2D/annotations/instances_inferencedense.json'
with open(dump_path, 'w') as f:
    json.dump(new_dict, f)

In [19]:
_COCO = COCO(dump_path)

print(len(_COCO.imgs))
print(len(_COCO.anns))
print(len(_COCO.cats))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
200
4001
4
